# Chapter 4  - Jinja Templates in Django

## Jinja Template Configuration in Django

In [1]:
import os
import jinja2


 4.1. Jinja configuration in Django settings.py

In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
PROJECT_DIR = os.path.dirname(os.path.abspath(__file__))
TEMPLATES = [
    {
        'BACKEND':'django.template.backends.jinja2.Jinja2',
        'DIRS': ['%s/jinjatemplates/'% (PROJECT_DIR),],
        'APP_DIRS': True,
        #...
    },
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        #...
    },
]


4.2. Django apps with jinja2 dirs with potential conflict and namespace qualification



Templates directly under jinja2 folder can cause loading conflicts


```
+---+-<PROJECT_DIR_project_name_conflict>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-about(app)-+
    |            +-__init__.py
    |            +-models.py
    |            +-tests.py
    |            +-views.py
    |            +-jinja2-+
    |                     |
    |                     +-index.html
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-jinja2-+
                           |
                           +-index.html
```


Templates classified with additional namespace avoid loading conflicts


```
+---+-<PROJECT_DIR_project_name_namespace>
|
+-__init__.py
+-settings.py
+-urls.py
+-wsgi.py
|
+-about(app)-+
|            +-__init__.py
|            +-models.py
|            +-tests.py
|            +-views.py
|            +-jinja2-+
|                     |
|                     +-about-+
|                             |
|                             +-index.html
+-stores(app)-+
              +-__init__.py
              +-models.py
              +-tests.py
              +-views.py
              +-jinja2-+
                       |
                       +-stores-+
                                |
                                +-index.htm
```


4.3. Jinja disable auto-escaping in Django


In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'autoescape': False
        },
    }
]

4.4. Generate error for invalid variables in Jinja with jinja2.StrictUndefined

In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'undefined':jinja2.StrictUndefined
        },
    }
]

## Create Reusable Jinja Templates

4.5. Jinja template with {% block %} tags


In [ ]:
<!DOCTYPE html>
    <html lang="en">
        <head>
            <meta charset="utf-8">
            <title>{% block title%}Default title{% endblock title %}</title>
            <meta name="description" content="{% block metadescription%}{% endblock metadescription %}">
            <meta name="keywords" content="{% block metakeywords%}{% endblock metakeywords %}">
            <!--......-->


4.6. Jinja template with {% extends %} and {% block %} tag

In [ ]:
{% if user %}
    {% extends "base.html" %}
{% else %}
    {% extends "signup_base.html" %}
{% endif %}

{% block title %}Coffeehouse home page{% endblock %}


4.7. Jinja templates use of super() with three reusable templates


In [ ]:
# base.html template

<p>{% block breadcrumb %}Home{% endblock %}</p>

# index.html template

{% extends "base.html" %}
{% block breadcrumb %}Main{% endblock %}

# detail.html template

{% extends "index.html" %}
{% block breadcrumb %} {{super()}} : Detail {% endblock %}


4.8. Jinja {% macro %} definition and use of {% import %}


In [ ]:
# base.html template

{% macro coffeestore(name, id='', address='', city='San Diego', state='CA', email=None) -%}
    <a id="{{id}}"></a>
    <h4>{{name}}</h4>
    <p>{{address}} {{city}},{{state}}</p>
    {% if email %}
        <p><a href='mailto:{{email}}'>{{email}}</a></p>
    {% endif %}
{%- endmacro %}

# index.html template calls inherited macro directly

{% extends "base.html" %}
{{coffeestore('Downtown', 1, 'Horton Plaza', 'San Diego', 'CA', 'downtown@coffeehouse.com')}}

# detail.html template with no extends, uses {% import %} to access macro in base.html

{% import 'base.html' as base %}
{{base.coffeestore('Downtown', 1, 'Horton Plaza', 'San Diego', 'CA', 'downtown@coffeehouse.com')}}

# otherdetail.html template with no extends, uses {% from import %} to access macro in base.html

{% from 'base.html' import coffeestore as mycoffeestoremacro %}
{{mycoffeestoremacro('Downtown', 1, 'Horton Plaza', 'San Diego', 'CA', 'downtown@coffeehouse.com')}}


4.9. Jinja {% call %} and {% macro %} use


In [ ]:
# macro definition

{% macro contentlist(adcolumn_width=3,contentcolumn_width=6) -%}
    <div class="col-md-{{adcolumn_width}}">Sidebar ads</div>
    <div class="col-md-{{contentcolumn_width}}">{{ caller() }}</div>
    <div class="col-md-{{adcolumn_width}}">Sidebar ads</div>
{%- endmacro %}

# macro call/invocation

{% call contentlist() %}
    <ul>
        <li>This is my list</li>
    </ul>
{% endcall %}

# rendering

<div class="col-md-3">Sidebar ads</div>
<div class="col-md-6">
    <ul>
        <li>This is my list</li>
    </ul>
</div>
<div class="col-md-3">Sidebar ads</div>
    

4.10. Jinja {% call %} and {% macro %} recursive calls


In [ ]:
# macro definition

{% macro contentlist(itemlist, adcolumn_width=3, contentcolumn_width=6) -%}
    <div class="col-md-{{adcolumn_width}}">Sidebar ads</div>
    <div class="col-md-{{contentcolumn_width}}">
        {% for item in itemlist %}
            {{ caller(item) }}
        {% endfor %}
    </div>
    <div class="col-md-{{adcolumn_width}}">Sidebar ads</div>
{%- endmacro %}

# variable definition

{% set coffeestores = [
    {
        'id': 0,
        'name':'Corporate',
        'address':'624 Broadway',
        'city':'San Diego',
        'state':'CA',
        'email':'corporate@coffeehouse.com'
    },
    {
        'id':1,
        'name':'Downtown',
        'address':'Horton Plaza',
        'city':'San Diego',
        'state':'CA',
        'email':'downtown@coffeehouse.com'
    },
    {
        'id':2,
        'name':'Uptown',
        'address':'1240 University Ave',
        'city':'San Diego',
        'state':'CA',
        'email':'uptown@coffeehouse.com'
    },
    {
        'id':3,
        'name':'Midtown',
        'address':'784 W Washington St',
        'city':'San Diego',
        'state':'CA',
        'email':'midtown@coffeehouse.com'
    }
]%}
    
# macro call/invocation

{% call(item) contentlist(coffeestores) %}
    <a id="{{item.id}}"></a>
    <h4>{{item.name}}</h4>
    <p>{{item.address}} {{item.city}}, {{item.state}}</p>
    {% if item.email %}<p><a href='mailto:{{item.email}}'>{{item.email}}</a></p>{% endif %}
{% endcall %}

# rendering

<div class="col-md-3">Sidebar ads</div>
<div class="col-md-6">
    <a id="0"></a>
    <h4>Corporate</h4>
    <p>624 Broadway San Diego, CA</p>
    <p><a href="mailto:corporate@coffeehouse.com">corporate@coffeehouse.com</a></p>

    <a id="1"></a>
    <h4>Downtown</h4>
    <p>Horton Plaza San Diego, CA</p>
    <p><a href="mailto:downtown@coffeehouse.com">downtown@coffeehouse.com</a></p>

    <a id="2"></a>
    <h4>Uptown</h4>
    <p>1240 University Ave San Diego, CA</p>
    <p><a href="mailto:uptown@coffeehouse.com">uptown@coffeehouse.com</a></p>

    <a id="3"></a>
    <h4>Midtown</h4>
    <p>784 W Washington St San Diego, CA</p>
    <p><a href="mailto:midtown@coffeehouse.com">midtown@coffeehouse.com</a></p>
</div>
<div class="col-md-3">Sidebar ads</div>


## Jinja Globals

In [ ]:
from jinja2.environment import Environment
from django.contrib.staticfiles.storage import staticfiles_storage
from django.core.urlresolvers import reverse


4.11. Custom Jinja environment with global variable


In [ ]:
class JinjaEnvironment(Environment):
    def __init__(self, **kwargs):
        super(JinjaEnvironment, self).__init__(**kwargs)
        self.globals['static'] = staticfiles_storage.url
        self.globals['reverse'] = reverse

4.12. Configure custom Jinja environment in Django setttings.py


In [ ]:
TEMPLATES = [
    {
        'BACKEND':'django.template.backends.jinja2.Jinja2',
        'DIRS': ['%s/templates/'% (PROJECT_DIR),],
        'APP_DIRS': True,
        'OPTIONS': {
            'environment': 'coffeehouse.jinja.env.JinjaEnvironment'
        }
    },
]

4.13. Directory structure and location of custom Jinja environment


```
+---+-<PROJECT_DIR_coffeehouse>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-jinja-+
            +-__init__.py
            +-env.py
```

## Jinja Built-In Statements/Tags and Functions

4.14. Jinja {% if %} statement with {% elif %} and {% else %}


In [ ]:
{% if drinks %}
     We have drinks!
{% endif %}

{% if drinks %}
    We have drinks
{% else %}
    No drinks,sorry
{% endif %}

{% if drinks %}
    We have drinks
{% elif drinks_on_sale %}
    We have drinks on sale!
{% else %}
    No drinks, sorry
{% endif %}

4.15. Jinja {% for %} statement and {% for %} with {% else %}

In [ ]:
<ul>
{% for drink in drinks %}
    <li>{{ drink.name }}</li>
{% else %}
    <li>No drinks, sorry</li>
{% endfor %}
</ul>


<ul> 
{% for storeid,store in stores %}
    <li>
        <a href="/stores/{{storeid}}/">{{store.name}}</a>
    </li>
{% endfor %}
</ul>



4.16. Jinja {% for %} statement with recursive keyword


In [ ]:
# Dictionary definition
coffees = {
    'espresso': {
        'nothing else': 'Espresso',
        'water': 'Americano',
        'steamed milk': {
            'more steamed milk than milk foam': 'Latte',
            'chocolate syrup': {'Whipped cream': 'Mocha'}
        },
        'more milk foam than steamed milk': 'Capuccino'
    }
}

#  Template definition with for and recursive
{% for ingredient, result in coffees.iteritems() recursive %}
    <li>
    {{ ingredient}}
    {% if result is mapping %}
        <ul>{{ loop(result.iteritems()) }}</ul>
    {% else %}
        YOU GET: {{ result }}
    {% endif %}
    </li>
{% endfor %}

# Output
espresso
    water YOU GET: Americano
    steamed milk
        more steamed milk than milk foam YOU GET: Latte
        chocolate syrup
            Whipped cream YOU GET: Mocha
    more milk foam than steamed milk YOU GET: Capuccino
    nothing else YOU GET: Espresso


4.17. Jinja cycler function

In [ ]:
{% set row_class = cycler('white', 'lightgrey', 'grey') %}

<ul>
{% for item in items %}
    <li class="{{ row_class.next() }}">{{ item }}</li>
{% endfor %}

{% for otheritem in moreitems %}
    <li class="{{ row_class.next() }}">{{ otheritem }}</li>
{% endfor %}

# Output
<ul>
    <li class="white">Item 1</li>
    <li class="lightgrey">Item 2 </li>
    <li class="grey">Item 3 </li>
    <li class="white">Item 4</li>
    <li class="lightgrey">Item 5</li>
    <li class="grey">Other item 1</li>
    <li class="white">Other item 2</li>
</ul>


4.18. Jinja joiner function


In [ ]:
{% set slash_joiner = joiner("/ ") %}
User: 
{% if username %} 
    {{ slash_joiner() }} {{username}}
{% endif %}

{% if alias %} 
    {{ slash_joiner() }} {{alias}}
{% endif %}

{% if nickname %} 
    {{ slash_joiner() }} {{nickname}}
{% endif %}

# Output
# If all variables are defined
User: username/alias/nickname

# If only nickname is defined
User: nickname

# If only username and alias is defined
User: username/alias

# Etc, the joiner function avoids any unnecessary preceding slash 
# because it doesn't print anything the first time its called


## Jinja Built-In Filters and Tests

4.19. Jinja defined test


In [ ]:
{% if variable is defined %}
    value of variable: {{ variable }}
{% else %}
    variable is not defined
{% endif %}

4.20. Jinja groupby filter

In [ ]:
# Dictionary definition
stores = [
    {'name': 'Downtown', 'street': '385 Main Street', 'city': 'San Diego'},
    {'name': 'Uptown', 'street': '231 Highland Avenue', 'city': 'San Diego'},
    {'name': 'Midtown', 'street': '85 Balboa Street', 'city': 'San Diego'},
    {'name': 'Downtown', 'street': '639 Spring Street', 'city': 'Los Angeles'},
    {'name': 'Midtown', 'street': '1407 Broadway Street', 'city': 'Los Angeles'},
    {'name': 'Downton', 'street': '50 1st Street', 'city': 'San Francisco'},
]

<ul>
{% for group in stores|groupby('city') %}
    <li>{{ group.grouper }}
    <ul>
        {% for item in group.list %}
            <li>{{ item.name }}: {{ item.street }}</li>
        {% endfor %}
    </ul>
    </li>
{% endfor %}
</ul>

# Output
Los Angeles
    Downtown: 639 Spring Street
    Midtown: 1407 Broadway Street
San Diego
    Downtown : 385 Main Street
    Uptown : 231 Highland Avenue
    Midtown : 85 Balboa Street
San Francisco
    Downtown: 50 1st Street

4.21. Jinja wordwrap filter

In [ ]:
# Variable definition

Coffeehouse started as a small store

# Template definition with wordwrap filter for every 12 characters

{{variable|wordwrap(12)}}

# Output

Coffeehouse
started as a
small store

4.22. Django xmlattr filter

In [ ]:
# Variable definition
{% set stores = [
    {'id':123, 'name': 'Downtown', 'street': '385 Main Street', 'city': 'San Diego'},
    {'id':243, 'name': 'Uptown', 'street': '231 Highland Avenue', 'city': 'San Diego'},
    {'id':357, 'name': 'Midtown', 'street': '85 Balboa Street', 'city': 'San Diego'},
    {'id':478, 'name': 'Downtown', 'street': '639 Spring Street', 'city': 'Los Angeles'},
    {'id':529, 'name': 'Midtown', 'street': '1407 Broadway Street', 'city': 'Los Angeles'},
    {'id':653, 'name': 'Downton', 'street': '50 1st Street', 'city': 'San Francisco'},
] %}

# Template definition
<ul>
{% for store in stores %}
    <li {{ 
        {
            'id':'%d'| format(store.id), 'class':'%s' | 
            format(store.city|lower|replace(' ','-')) 
        } | xmlattr 
    }}> 
        {{store.city}} {{store.name}}
    </li>
{% endfor %}
</ul>

# Output
<ul>
    <li id="123" class="san-diego"> San Diego Downtown</li>
    <li id="243" class="san-diego"> San Diego Uptown</li>
    <li id="357" class="san-diego"> San Diego Midtown</li>
    <li id="478" class="los-angeles"> Los Angeles Downtown</li>
    <li id="529" class="los-angeles"> Los Angeles Midtown</li>
    <li id="653" class="san-francisco"> San Francisco Downton</li>
</ul>

## Custom Filters and Tests in Jinja

In [1]:
import math
import jinja2

4.23. Backing Python methods for Jinja custom filters and tests.

In [ ]:
def customcoffee(value, arg="muted"):
    return jinja2.Markup('%s' % (arg, value))

def squarerootintext(value):
    return "The square root of %s is %s" % (value, math.sqrt(value))

def startswithvowel(value):
    if value.lower().startswith(("a", "e", "i", "o", "u")):
        return True
    else:
        return False

4.24. Directory structure and location of custom Jinja filters and tests


```
+---+-<PROJECT_DIR_coffeehouse>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-jinja-+
            +-__init__.py
            +-env.py
            +-filters.py
```

4.25. Custom Jinja environment with custom filters and tests


In [ ]:
from jinja2.environment import Environment
from coffeehouse.jinja.filters import customcoffee, squarerootintext, startswithvowel


class JinjaEnvironment(Environment):
    def __init__(self, **kwargs):
        super(JinjaEnvironment, self).__init__(**kwargs)
        self.filters['customcoffee'] = customcoffee
        self.filters['squarerootintext'] = squarerootintext
        self.filters['startswithvowel'] = startswithvowel
        self.tests['startswithvowel'] = startswithvowel



4.26. Configure custom Jinja environment in Django setttings.py


In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'environment': 'coffeehouse.jinja.env.JinjaEnvironment'
        }
    },
]

4.27. Jinja extension configuration in Django


In [ ]:
TEMPLATES = [
    {
        'BACKEND':'django.template.backends.jinja2.Jinja2',
        #...
        'OPTIONS': {
            'extensions': [
                'jinja2.ext.loopcontrols',
                'jdj_tags.extensions.DjangoCompat',
                'coffeehouse.jinja.extensions.DjangoNow',
            ],
        }
    }
]

## Jinja Extensions

In [ ]:
from jinja2 import lexer, nodes
from jinja2.ext import Extension
from django.utils import timezone
from django.template.defaultfilters import date
from django.conf import settings
from datetime import datetime


4.28. Jinja custom extension for Jinja {% now %} statement


In [ ]:
class DjangoNow(Extension):
    tags = set(['now'])

    def _now(self, date_format):
        tzinfo = timezone.get_current_timezone() if settings.USE_TZ else None
        formatted = date(datetime.now(tz=tzinfo), date_format)
        return formatted

    def parse(self, parser):
        lineno = next(parser.stream).lineno
        token = parser.stream.expect(lexer.TOKEN_STRING)
        date_format = nodes.Const(token.value)
        call = self.call_method('_now', [date_format], lineno=lineno)
        token = parser.stream.current
        if token.test('name:as'):
            next(parser.stream)
            as_var = parser.stream.expect(lexer.TOKEN_NAME)
            as_var = nodes.Name(as_var.value, 'store', lineno=as_var.lineno)
            return nodes.Assign(as_var, call, lineno=lineno)
        else:
            return nodes.Output([call], lineno=lineno)


4.29. Directory structure and location of custom Jinja extension


```
+---+-<PROJECT_DIR_coffeehouse>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-jinja-+
            +-__init__.py
            +-extensions.py
```

4.30. Custom Jinja environment with policies

In [ ]:
from jinja2.environment import Environment
from coffeehouse.jinja.filters import customcoffee, squarerootintext, startswithvowel


class JinjaEnvironment(Environment):
    def __init__(self,**kwargs):
        super(JinjaEnvironment, self).__init__(**kwargs)
        self.policies['truncate.leeway'] = 0